In [1]:
import matplotlib.pyplot as plt
import cv2
import json
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from progressbar import ProgressBar
import time
from multiprocessing import Pool
from functools import partial

from sklearn.model_selection import train_test_split
from torchvision.utils import make_grid
import gc

In [2]:
np.random.seed(1337)


In [3]:
torch.__version__

'2.1.0.dev20230317+cu118'

In [4]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        #x = torch.Tensor(self.features[idx])
        #y = torch.Tensor(self.targets[idx])
        x = self.X[idx]
        y = self.y[idx]
        
        return x,y

In [5]:
def show_statistics(VALID_images,VALID_depth_images,MIDAIR_images,MIDAIR_depth_images):

    
    #loading both dataset into seperate dataset classes
    dataset1 = MyDataset(VALID_images, VALID_depth_images)
    dataset2 = MyDataset(MIDAIR_images, MIDAIR_depth_images)
    
    # concatenating training, validation and testing images into one tensor
    X_combined = torch.cat((dataset1.X,dataset2.X),dim=0)
    y_combined = torch.cat((dataset1.y,dataset2.y),dim=0)
    
    for channel in range(X_combined.shape[1]):
        values_in_channel = X_combined[:,channel,:,:]
        min_value = torch.min(values_in_channel)
        max_value = torch.max(values_in_channel)
        mean_value = torch.mean(values_in_channel.float())
        std_value = torch.std(values_in_channel)
        print(f"X-data channel {channel+1}")
        print(f"  Min: {min_value.item()}")
        print(f"  Max: {max_value.item()}")
        print(f"  Mean: {mean_value.item()}")
        print(f"  Standard Deviation: {std_value.item()}")
        X_combined[:,channel,:,:] = (values_in_channel-mean_value)/std_value
    print(torch.max(X_combined))   
    print("\n")
    
    for channel in range(y_combined.shape[1]):
        values_in_channel = y_combined[:,channel,:,:]
        min_value = torch.min(values_in_channel)
        max_value = torch.max(values_in_channel)
        mean_value = torch.mean(values_in_channel.float())
        std_value = torch.std(values_in_channel)
        print(f"y-data channel {channel+1}")
        print(f"  Min: {min_value.item()}")
        print(f"  Max: {max_value.item()}")
        print(f"  Mean: {mean_value.item()}")
        print(f"  Standard Deviation: {std_value.item()}")
        y_combined[:,channel,:,:] = (values_in_channel-mean_value)/std_value
        
    
   

In [6]:
def standardization(VALID_images,VALID_depth_images,MIDAIR_images,MIDAIR_depth_images):
    #loading both dataset into seperate dataset classes
    dataset1 = MyDataset(VALID_images, VALID_depth_images)
    dataset2 = MyDataset(MIDAIR_images, MIDAIR_depth_images)
    
     #splitting the dataset 
    X_train_VALID, X_temp_VALID, y_train_VALID, y_temp_VALID = train_test_split(dataset1.X, dataset1.y, test_size=0.3, random_state=44)
    X_val_VALID, X_test_VALID, y_val_VALID, y_test_VALID = train_test_split(X_temp_VALID,y_temp_VALID, test_size=0.5, random_state=44)
    
    X_train_MIDAIR, X_temp_MIDAIR, y_train_MIDAIR, y_temp_MIDAIR = train_test_split(dataset2.X, dataset2.y, test_size=0.3, random_state=44)
    X_val_MIDAIR, X_test_MIDAIR, y_val_MIDAIR, y_test_MIDAIR = train_test_split(X_temp_MIDAIR,y_temp_MIDAIR, test_size=0.5, random_state=44)
    
    X_combined_train = torch.cat((X_train_VALID, X_train_MIDAIR),dim=0)
    X_combined_val = torch.cat((X_val_VALID, X_val_MIDAIR),dim=0)
    
   
    y_combined_train = torch.cat((y_train_VALID, y_train_MIDAIR),dim=0)
    y_combined_val = torch.cat((y_val_VALID, y_val_MIDAIR),dim=0)
                               
    
    for channel in range(X_combined_train.shape[1]):
        values_in_channel = X_combined_train[:,channel,:,:]
        min_value = torch.min(values_in_channel)
        max_value = torch.max(values_in_channel)
        mean_value = torch.mean(values_in_channel.float())
        std_value = torch.std(values_in_channel)
        print(f"X-data channel {channel+1}")
        print(f"  Min: {min_value.item()}")
        print(f"  Max: {max_value.item()}")
        print(f"  Mean: {mean_value.item()}")
        print(f"  Standard Deviation: {std_value.item()}")
        X_combined_train[:,channel,:,:] = (X_combined_train[:,channel,:,:]-mean_value)/std_value                     
        X_combined_val[:,channel,:,:] = (X_combined_val[:,channel,:,:]-mean_value)/std_value
        X_test_VALID[:,channel,:,:] = (X_test_VALID[:,channel,:,:]-mean_value)/std_value
        X_test_MIDAIR[:,channel,:,:] = (X_test_MIDAIR[:,channel,:,:]-mean_value)/std_value
                               
                            
    
    print("\n")
    
    for channel in range(y_combined_train.shape[1]):
        values_in_channel = y_combined_train[:,channel,:,:]
        min_value = torch.min(values_in_channel)
        max_value = torch.max(values_in_channel)
        mean_value = torch.mean(values_in_channel.float())
        std_value = torch.std(values_in_channel)
        print(f"y-data channel {channel+1}")
        print(f"  Min: {min_value.item()}")
        print(f"  Max: {max_value.item()}")
        print(f"  Mean: {mean_value.item()}")
        print(f"  Standard Deviation: {std_value.item()}")
        y_combined_train[:,channel,:,:] = (y_combined_train[:,channel,:,:]-mean_value)/std_value                     
        y_combined_val[:,channel,:,:] = (y_combined_val[:,channel,:,:]-mean_value)/std_value
        y_test_VALID[:,channel,:,:] = (y_test_VALID[:,channel,:,:]-mean_value)/std_value
        y_test_MIDAIR[:,channel,:,:] = (y_test_MIDAIR[:,channel,:,:]-mean_value)/std_value
    

    
    return (X_combined_train.half(), y_combined_train.half()), (X_combined_val.half(), y_combined_val.half()), (X_test_VALID.half(), y_test_VALID.half()), (X_test_MIDAIR.half(), y_test_MIDAIR.half())

In [7]:
def standardization_normalization(VALID_images,VALID_depth_images,MIDAIR_images,MIDAIR_depth_images):
    #loading both dataset into seperate dataset classes
    dataset1 = MyDataset(VALID_images, VALID_depth_images)
    dataset2 = MyDataset(MIDAIR_images, MIDAIR_depth_images)
    
     #splitting the dataset 
    X_train_VALID, X_temp_VALID, y_train_VALID, y_temp_VALID = train_test_split(dataset1.X, dataset1.y, test_size=0.3, random_state=44)
    X_val_VALID, X_test_VALID, y_val_VALID, y_test_VALID = train_test_split(X_temp_VALID,y_temp_VALID, test_size=0.5, random_state=44)
    
    X_train_MIDAIR, X_temp_MIDAIR, y_train_MIDAIR, y_temp_MIDAIR = train_test_split(dataset2.X, dataset2.y, test_size=0.3, random_state=44)
    X_val_MIDAIR, X_test_MIDAIR, y_val_MIDAIR, y_test_MIDAIR = train_test_split(X_temp_MIDAIR,y_temp_MIDAIR, test_size=0.5, random_state=44)
    
    X_combined_train = torch.cat((X_train_VALID, X_train_MIDAIR),dim=0)
    X_combined_val = torch.cat((X_val_VALID, X_val_MIDAIR),dim=0)
    #HERE
    print("valid y",y_train_VALID.shape)
    print("midair y",y_train_MIDAIR.shape)
    y_combined_train = torch.cat((y_train_VALID, y_train_MIDAIR),dim=0)
    y_combined_val = torch.cat((y_val_VALID, y_val_MIDAIR),dim=0)
                               
    
    for channel in range(X_combined_train.shape[1]):
        values_in_channel = X_combined_train[:,channel,:,:]
        min_value = torch.min(values_in_channel)
        max_value = torch.max(values_in_channel)
        mean_value = torch.mean(values_in_channel.float())
        std_value = torch.std(values_in_channel)
        #print(f"X-data channel {channel+1}")
        #print(f"  Min: {min_value.item()}")
        #print(f"  Max: {max_value.item()}")
        #print(f"  Mean: {mean_value.item()}")
        #print(f"  Standard Deviation: {std_value.item()}")
        X_combined_train[:,channel,:,:] = (X_combined_train[:,channel,:,:]-mean_value)/std_value                     
        X_combined_val[:,channel,:,:] = (X_combined_val[:,channel,:,:]-mean_value)/std_value
        X_test_VALID[:,channel,:,:] = (X_test_VALID[:,channel,:,:]-mean_value)/std_value
        X_test_MIDAIR[:,channel,:,:] = (X_test_MIDAIR[:,channel,:,:]-mean_value)/std_value
                               
                            
    
    print("\n")
    
    

    min_value = torch.min(y_combined_train)
    max_value = torch.max(y_combined_train)
    y_combined_train = (y_combined_train-min_value)/(max_value-min_value)                   
    y_combined_val = (y_combined_val-min_value)/(max_value-min_value)
    y_test_VALID = (y_test_VALID-min_value)/(max_value-min_value)
    y_test_MIDAIR = (y_test_MIDAIR-min_value)/(max_value-min_value)


    
    return (X_combined_train.half(), y_combined_train.half()), (X_combined_val.half(), y_combined_val.half()), (X_test_VALID.half(), y_test_VALID.half()), (X_test_MIDAIR.half(), y_test_MIDAIR.half())

In [8]:
def min_max_feature_scaling(training_tensor, validation_tensor, testing_tensor_VALID, testing_tensor_MIDAIR):
    tensor_min = torch.min(training_tensor) 
    tensor_max = torch.max(training_tensor)
    
    training_tensor_scaled = (training_tensor-tensor_min)/(tensor_max-tensor_min)
    validation_tensor_scaled = (validation_tensor-tensor_min)/(tensor_max-tensor_min)
    
    testing_tensor_VALID_scaled = (testing_tensor_VALID-tensor_min)/(tensor_max-tensor_min)
    testing_tensor_MIDAIR_scaled = (testing_tensor_MIDAIR-tensor_min)/(tensor_max-tensor_min)
    #testing_tensor_scaled = (testing_tensor-tensor_min)/(tensor_max-tensor_min)
    
    return training_tensor_scaled.half(),validation_tensor_scaled.half(), testing_tensor_VALID_scaled.half(), testing_tensor_MIDAIR_scaled.half()

In [9]:
def normalization_and_standardization(VALID_images,VALID_depth_images,size, max_val=None):
#size = [small,xsmall,medium,large,xlarge]
    
    np.random.seed(1337)
    size = size.lower()
    
    if size == "tiny":
        l1,l2 = round(196*0.13134),196
    elif size == "xxsmall":
        l1,l2 = round(391*0.13134),391
    elif size == "xsmall":
        l1,l2 = round(782*0.13134),782
    elif size == "small":
        l1,l2 = round(1563*0.13134),1563
    elif size == "medium":
        l1,l2 = round(3125*0.13134),3125
    elif size == "large":
        l1,l2 = round(6250*0.13134),6250
    elif size == "xlarge":
        l1,l2 = round(12500*0.13134),12500
    elif size == "xxlarge":
        l1,l2 = round(25000*0.13134),25000
    elif size == "extreme":
        l1,l2 = 6567,50000
       
    #Check if folder exists
    if not os.path.exists(f"Fused_normalized_data/{size}") or not os.path.exists(f"Normalized_standardized_data_{max_val}_new/{size}"):
        #if not create folder
        
        #load images
        indicies = np.random.choice([i for i in range(6567)],size=l1,replace=False)
        VALID_images = torch.load(VALID_images)[indicies]
        VALID_depth_images = torch.load(VALID_depth_images)[indicies]
        MIDAIR_images = torch.load(f"C:/Users/Frede/Desktop/bachelorprojekt/MIDAIR_tensors/{l2}_images/MIDAIR_tensor_images.pt")
        MIDAIR_depth_images = torch.load(f"C:/Users/Frede/Desktop/bachelorprojekt/MIDAIR_tensors/{l2}_images/MIDAIR_tensor_depth_images_no_rgb.pt")
        print(MIDAIR_depth_images.shape)
        if max_val != None:
            if not os.path.exists(f"Normalized_standardized_data_{max_val}_new/{size}"):
                os.makedirs(f"Normalized_standardized_data_{max_val}_new/{size}")
                #clip datasets

                VALID_depth_images_np = np.clip(VALID_depth_images.numpy(), 0.001, max_val, VALID_depth_images.numpy())
                MIDAIR_depth_images_np = np.clip(MIDAIR_depth_images.numpy(), 0.001, max_val, MIDAIR_depth_images.numpy())

                VALID_depth_images = torch.from_numpy(VALID_depth_images_np)
                MIDAIR_depth_images = torch.from_numpy(MIDAIR_depth_images_np)

                #normalize datasets and combine them
                dataset = standardization_normalization(VALID_images, VALID_depth_images, MIDAIR_images, MIDAIR_depth_images)
                #index them and save them to folder
                X_train,y_train,X_val,y_val,X_test_VALID,y_test_VALID, X_test_MIDAIR, y_test_MIDAIR = dataset[0][0], dataset[0][1], dataset[1][0], dataset[1][1], dataset[2][0], dataset[2][1], dataset[3][0], dataset[3][1]
                torch.save(X_train, f"Normalized_standardized_data_{max_val}_new/{size}/X_train.pt")
                torch.save(y_train, f"Normalized_standardized_data_{max_val}_new/{size}/y_train.pt")
                torch.save(X_val, f"Normalized_standardized_data_{max_val}_new/{size}/X_val.pt")
                torch.save(y_val, f"Normalized_standardized_data_{max_val}_new/{size}/y_val.pt")
                torch.save(X_test_VALID, f"Normalized_standardized_data_{max_val}_new/{size}/X_test_VALID.pt")
                torch.save(y_test_VALID, f"Normalized_standardized_data_{max_val}_new/{size}/y_test_VALID.pt")
                torch.save(X_test_MIDAIR, f"Normalized_standardized_data_{max_val}_new/{size}/X_test_MIDAIR.pt")
                torch.save(y_test_MIDAIR, f"Normalized_standardized_data_{max_val}_new/{size}/y_test_MIDAIR.pt")

            else:
                VALID_depth_images_np = np.clip(VALID_depth_images.numpy(), 0.001, max_val, VALID_depth_images.numpy())
                MIDAIR_depth_images_np = np.clip(MIDAIR_depth_images.numpy(), 0.001, max_val, MIDAIR_depth_images.numpy())

                VALID_depth_images = torch.from_numpy(VALID_depth_images_np)
                MIDAIR_depth_images = torch.from_numpy(MIDAIR_depth_images_np)
               
                #normalize datasets and combine them
                dataset = standardization_normalization(VALID_images, VALID_depth_images, MIDAIR_images, MIDAIR_depth_images)
                #index them and save them to folder
                X_train,y_train,X_val,y_val,X_test_VALID,y_test_VALID, X_test_MIDAIR, y_test_MIDAIR = dataset[0][0], dataset[0][1], dataset[1][0], dataset[1][1], dataset[2][0], dataset[2][1], dataset[3][0], dataset[3][1]
                torch.save(X_train, f"Normalized_standardized_data_{max_val}_new/{size}/X_train.pt")
                torch.save(y_train, f"Normalized_standardized_data_{max_val}_new/{size}/y_train.pt")
                torch.save(X_val, f"Normalized_standardized_data_{max_val}_new/{size}/X_val.pt")
                torch.save(y_val, f"Normalized_standardized_data_{max_val}_new/{size}/y_val.pt")
                torch.save(X_test_VALID, f"Normalized_standardized_data_{max_val}_new/{size}/X_test_VALID.pt")
                torch.save(y_test_VALID, f"Normalized_standardized_data_{max_val}_new/{size}/y_test_VALID.pt")
                torch.save(X_test_MIDAIR, f"Normalized_standardized_data_{max_val}_new/{size}/X_test_MIDAIR.pt")
                torch.save(y_test_MIDAIR, f"Normalized_standardized_data_{max_val}_new/{size}/y_test_MIDAIR.pt")
            return X_train, X_val, y_train, y_val, X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_MIDAIR 
        
            
VALID_images_path = r"C:/Users/Frede/Desktop/bachelorprojekt/VALID_tensors/VALID_tensor_images.pt"
VALID_depth_images_path_no_rgb = r"C:/Users/Frede/Desktop/bachelorprojekt/VALID_tensors/VALID_tensor_depth_images_no_rgb.pt"



sizes = ["tiny","xxsmall"]
#sizes = ["extreme"]
for s in sizes:
    print("Initiating ", s)
    normalization_and_standardization(VALID_images_path,VALID_depth_images_path_no_rgb,s,80)
    print("Finished", s)

Initiating  tiny
torch.Size([180, 1, 256, 256])
valid y torch.Size([18, 1, 256, 256])
midair y torch.Size([126, 1, 256, 256])


Finished tiny
Initiating  xxsmall
torch.Size([360, 1, 256, 256])
valid y torch.Size([35, 1, 256, 256])
midair y torch.Size([252, 1, 256, 256])


Finished xxsmall


In [33]:
def Standardization_only(VALID_images,VALID_depth_images,size, max_val=None):
#size = [small,xsmall,medium,large,xlarge]
    

    size = size.lower()

    if size == "xsmall":
        l1,l2 = 500,500
    elif size == "small":
        l1,l2 = 1000,1000
    elif size == "medium":
        l1,l2 = 2500,2500
    elif size == "large":
        l1,l2 = 6567,7500
    elif size == "xlarge":
        l1,l2 = 6567,12500
    elif size == "xxlarge":
        l1,l2 = 6567,25000
    elif size == "extreme":
        l1,l2 = 6567,50000
       
    #Check if folder exists
    if not os.path.exists(f"Standardized_data/{size}") or not os.path.exists(f"Standardized_data_{max_val}/{size}"):
        #if not create folder
        
        #load images
        VALID_images = torch.load(VALID_images)[0:l1]
        VALID_depth_images = torch.load(VALID_depth_images)[0:l1]
        MIDAIR_images = torch.load(f"C:/Users/Frede/Desktop/bachelorprojekt/MIDAIR_tensors/{l2}_images/MIDAIR_tensor_images.pt")
        MIDAIR_depth_images = torch.load(f"C:/Users/Frede/Desktop/bachelorprojekt/MIDAIR_tensors/{l2}_images/MIDAIR_tensor_depth_images_no_rgb.pt")
        if max_val != None:
            if not os.path.exists(f"Standardized_data_{max_val}/{size}"):
                os.makedirs(f"Standardized_data_{max_val}/{size}")
                #clip datasets

                VALID_depth_images_np = np.clip(VALID_depth_images.numpy(), 0.001, max_val, VALID_depth_images.numpy())
                MIDAIR_depth_images_np = np.clip(MIDAIR_depth_images.numpy(), 0.001, max_val, MIDAIR_depth_images.numpy())

                VALID_depth_images = torch.from_numpy(VALID_depth_images_np)
                MIDAIR_depth_images = torch.from_numpy(MIDAIR_depth_images_np)

                #normalize datasets and combine them
                dataset = standardization(VALID_images, VALID_depth_images, MIDAIR_images, MIDAIR_depth_images)
                #index them and save them to folder
                X_train,y_train,X_val,y_val,X_test_VALID,y_test_VALID, X_test_MIDAIR, y_test_MIDAIR = dataset[0][0], dataset[0][1], dataset[1][0], dataset[1][1], dataset[2][0], dataset[2][1], dataset[3][0], dataset[3][1]
                torch.save(X_train, f"Standardized_data_{max_val}/{size}/X_train.pt")
                torch.save(y_train, f"Standardized_data_{max_val}/{size}/y_train.pt")
                torch.save(X_val, f"Standardized_data_{max_val}/{size}/X_val.pt")
                torch.save(y_val, f"Standardized_data_{max_val}/{size}/y_val.pt")
                torch.save(X_test_VALID, f"Standardized_data_{max_val}/{size}/X_test_VALID.pt")
                torch.save(y_test_VALID, f"Standardized_data_{max_val}/{size}/y_test_VALID.pt")
                torch.save(X_test_MIDAIR, f"Standardized_data_{max_val}/{size}/X_test_MIDAIR.pt")
                torch.save(y_test_MIDAIR, f"Standardized_data_{max_val}/{size}/y_test_MIDAIR.pt")

            else:
                VALID_depth_images_np = np.clip(VALID_depth_images.numpy(), 0.001, max_val, VALID_depth_images.numpy())
                MIDAIR_depth_images_np = np.clip(MIDAIR_depth_images.numpy(), 0.001, max_val, MIDAIR_depth_images.numpy())

                VALID_depth_images = torch.from_numpy(VALID_depth_images_np)
                MIDAIR_depth_images = torch.from_numpy(MIDAIR_depth_images_np)

                #normalize datasets and combine them
                dataset = standardization(VALID_images, VALID_depth_images, MIDAIR_images, MIDAIR_depth_images)
                #index them and save them to folder
                X_train,y_train,X_val,y_val,X_test_VALID,y_test_VALID, X_test_MIDAIR, y_test_MIDAIR = dataset[0][0], dataset[0][1], dataset[1][0], dataset[1][1], dataset[2][0], dataset[2][1], dataset[3][0], dataset[3][1]
                torch.save(X_train, f"Standardized_data_{max_val}/{size}/X_train.pt")
                torch.save(y_train, f"Standardized_data_{max_val}/{size}/y_train.pt")
                torch.save(X_val, f"Standardized_data_{max_val}/{size}/X_val.pt")
                torch.save(y_val, f"Standardized_data_{max_val}/{size}/y_val.pt")
                torch.save(X_test_VALID, f"Standardized_data_{max_val}/{size}/X_test_VALID.pt")
                torch.save(y_test_VALID, f"Standardized_data_{max_val}/{size}/y_test_VALID.pt")
                torch.save(X_test_MIDAIR, f"Standardized_data_{max_val}/{size}/X_test_MIDAIR.pt")
                torch.save(y_test_MIDAIR, f"Standardized_data_{max_val}/{size}/y_test_MIDAIR.pt")
            return X_train, X_val, y_train, y_val, X_test_VALID, y_test_VALID, X_test_MIDAIR, y_test_MIDAIR
        
            
VALID_images_path = r"C:/Users/Frede/Desktop/bachelorprojekt/VALID_tensors/VALID_tensor_images.pt"
VALID_depth_images_path_no_rgb = r"C:/Users/Frede/Desktop/bachelorprojekt/VALID_tensors/VALID_tensor_depth_images_no_rgb.pt"



#sizes = ["xsmall","small","medium","large", "xlarge", "xxlarge"]
#sizes = ["extreme"]
for s in sizes:
    Standardization_only(VALID_images_path,VALID_depth_images_path_no_rgb,s,80)


valid y torch.Size([350, 1, 256, 256])
midair y torch.Size([378, 1, 256, 256])
X-data channel 1
  Min: 0.0
  Max: 255.0
  Mean: 57.44839096069336
  Standard Deviation: 66.3125
X-data channel 2
  Min: 0.0
  Max: 255.0
  Mean: 87.5497817993164
  Standard Deviation: 67.1875
X-data channel 3
  Min: 0.0
  Max: 255.0
  Mean: 90.70647430419922
  Standard Deviation: 69.8125


y-data channel 1
  Min: 1.5166015625
  Max: 80.0
  Mean: 62.861209869384766
  Standard Deviation: 27.65625
